# CM4 velocity magnitude across Solomon Strait 
*Akira Di Sandro, 6/15/20*
<br>
In this notebook, I will be attempting to calculate the velocity magnitude across the cross-section of the Solomon Sea.

## 1. Import Python Packages

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [15]:
import xarray as xr
xr.set_options(display_style='html')
import intake
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np

## 2. Import CM4 data from CMIP6 dataset

In [16]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)

In [17]:
dog = col.search(source_id='GFDL-CM4', experiment_id='historical', table_id='Omon', variable_id=['uo','vo','wo'], grid_label='gn')

In [18]:
dset_dict = dog.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

Dataset(s):   0%|                                        | 0/1 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


Dataset(s): 100%|████████████████████████████████| 1/1 [00:02<00:00,  2.91s/it]


['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']

In [19]:
CM4_hist = dset_dict['CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn']
CM4_hist

<xarray.Dataset>
Dimensions:    (bnds: 2, lev: 35, member_id: 1, time: 1980, x: 1440, y: 1080)
Coordinates:
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
    lat        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * bnds       (bnds) float64 1.0 2.0
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x          (x) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * y          (y) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * member_id  (member_id) <U8 'r1i1p1f1'
Data variables:
    uo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
    vo         (member_id, time, lev, y, x) float32 dask.array<chunksize=(1, 1, 35, 1080, 1440), meta=np.ndarray>
Attributes:
    history:                 File was processed by fremetar (GFDL analog of C...
    grid:                    ocean data on native grid tripolar - nominal 0.2...
    title:                   NOAA GFDL GFDL-CM4 model output prepared for CMI...
    data_specs_version:      01.00.27
    product:                 model-output
    sub_experiment:          none
    grid_label:              gn
    nominal_resolution:      25 km
    initialization_index:    1
    parent_experiment_id:    piControl
    realm:                   ocean
    activity_id:             CMIP
    frequency:               mon
    institution_id:          NOAA-GFDL
    source_type:             AOGCM
    experiment:              historical
    branch_time_in_child:    0.0
    institution:             National Oceanic and Atmospheric Administration,...
    further_info_url:        https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.G...
    parent_activity_id:      CMIP
    physics_index:           1
    realization_index:       1
    source:                  GFDL-CM4 (2018): \naerosol: interactive\natmos: ...
    parent_mip_era:          CMIP6
    comment:                 <null ref>
    status:                  2019-08-06;created;by nhn2@columbia.edu
    Conventions:             CF-1.7 CMIP-6.0 UGRID-1.0
    references:              see further_info_url attribute
    contact:                 gfdl.climate.model.info@noaa.gov
    forcing_index:           1
    table_id:                Omon
    parent_time_units:       days since 0001-1-1
    branch_time_in_parent:   36500.0
    tracking_id:             hdl:21.14100/d0ebcd36-d2cd-442e-a8d9-8f7a27168a2...
    experiment_id:           historical
    license:                 CMIP6 model data produced by NOAA-GFDL is licens...
    variant_label:           r1i1p1f1
    mip_era:                 CMIP6
    source_id:               GFDL-CM4
    sub_experiment_id:       none
    parent_source_id:        GFDL-CM4
    parent_variant_label:    r1i1p1f1
    branch_method:           standard
    variant_info:            N/A
    intake_esm_dataset_key:  CMIP.NOAA-GFDL.GFDL-CM4.historical.Omon.gn

* Sidenote: I just realized that taking the magnitude of the velocity components at this cross section is a little meaningless because the cross-section is along a line of latitude, meaning the 'uo' or u-component (zonal) velocity will add nothing to the magnitude (sin(0)=0) while the 'vo' or v-component (meridional) velocity will be the magnitude itself (sine(pi/2)=1).

* This means I need to first figure out a way to make a diagonal cross-section of the Solomon Sea (or maybe directly of the Solomon Strati).

## 3. Make Function to find Magnitude at every (x,y)

In [20]:
# new projection/transformation based on check-in
def calcmag(vel, theta):
    mat = [[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]
    proj_vel = np.dot(vel, mat)
    return proj_vel
#theta above x-axis is positive, below is negative

### a. Testing out the function

Defining test variables

Testing out the calcmag function

In [22]:
svel = [1,1]
theta = np.pi/2

spvel = calcmag(svel,theta)
spvel

array([ 1., -1.])

### b. Using the function on our dataset

## 4. Make Diagonal Cross-section 

In [20]:
from dask.distributed import Client
from dask_gateway import Gateway


gateway = Gateway()  # connect to Gateway

cluster = gateway.new_cluster()  # create cluster
cluster.scale(10)  # scale cluster

client = Client(cluster)  # connect Client to Cluster

In [21]:
client

Client Scheduler: gateway://traefik-ocean-prod-dask-gateway.ocean-prod:80/ocean-prod.a155500167764ed193b0e38116f84b3f Dashboard: https://ocean.pangeo.io/services/dask-gateway/clusters/ocean-prod.a155500167764ed193b0e38116f84b3f/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [13]:
diag_cross_u = CM4_hist['uo'].sel(time='2012').mean(dim='time').sel(y=slice(-6,-4)).sel(x=slice(-208,-206))
diag_cross_u

<xarray.DataArray 'uo' (lev: 35, y: 8, x: 8)>
dask.array<getitem, shape=(35, 8, 8), dtype=float32, chunksize=(35, 8, 8), chunktype=numpy.ndarray>
Coordinates:
    lat        (y, x) float32 dask.array<chunksize=(8, 8), meta=np.ndarray>
    lon        (y, x) float32 dask.array<chunksize=(8, 8), meta=np.ndarray>
  * x          (x) float64 -208.0 -207.7 -207.5 -207.2 ... -206.7 -206.5 -206.2
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * y          (y) float64 -5.989 -5.74 -5.492 -5.243 ... -4.745 -4.495 -4.246
    member_id  <U8 'r1i1p1f1'

### Closing the Cluster
Make sure to **ALWAYS** close the cluster and client, so I'm not wasting processing tools on simple calculations and tasks.

In [23]:
client.close()
cluster.close()